### Problem Data

In [4]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [5]:
time_req = {
    "Akron": {"Compact":80, "Coupe": 56, "Sedan":43, "SUV":62, "Truck":46, "Van":58},
    "Buffalo": {"Compact":94, "Coupe": 50, "Sedan":88, "SUV":64, "Truck":63, "Van":52},
    "Columbus": {"Compact":94, "Coupe": 46, "Sedan":50, "SUV":40, "Truck":55, "Van":73},
    "Detroit": {"Compact":98, "Coupe": 79, "Sedan":71, "SUV":65, "Truck":91, "Van":59},
    "Evansville": {"Compact":61, "Coupe": 59, "Sedan":89, "SUV":98, "Truck":45, "Van":52},
    "Flint": {"Compact":77, "Coupe": 49, "Sedan":65, "SUV":95, "Truck":72, "Van":91}
}

In [3]:
plants = ["Akron","Buffalo","Columbus","Detroit","Evansville","Flint"]
products = ["Compact","Coupe","Sedan","SUV","Truck","Van"]

In [6]:
model = gp.Model('Production Assignment')

Restricted license - for non-production use only - expires 2024-10-28


In [11]:
# decision variable
plant_make_product = model.addVars(plants, products, vtype=GRB.BINARY, name="Plant Production")
print(plant_make_product)

{('Akron', 'Compact'): <gurobi.Var *Awaiting Model Update*>, ('Akron', 'Coupe'): <gurobi.Var *Awaiting Model Update*>, ('Akron', 'Sedan'): <gurobi.Var *Awaiting Model Update*>, ('Akron', 'SUV'): <gurobi.Var *Awaiting Model Update*>, ('Akron', 'Truck'): <gurobi.Var *Awaiting Model Update*>, ('Akron', 'Van'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Compact'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Coupe'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Sedan'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'SUV'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Truck'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Van'): <gurobi.Var *Awaiting Model Update*>, ('Columbus', 'Compact'): <gurobi.Var *Awaiting Model Update*>, ('Columbus', 'Coupe'): <gurobi.Var *Awaiting Model Update*>, ('Columbus', 'Sedan'): <gurobi.Var *Awaiting Model Update*>, ('Columbus', 'SUV'): <gurobi.Var *Awaiting Model Update*>, ('Columbus', 'Truck'): <gurobi.Var *Awaiting Model 

In [14]:
# objective function
model.setObjective(gp.quicksum(plant_make_product[plant, product]*time_req[plant][product]
                 for plant in plants for product in products), GRB.MINIMIZE)

In [23]:
# constraints
# Note: quicksum should be wrapped with parantheses
plant_make = model.addConstrs((gp.quicksum(plant_make_product[plant, product]
                                         for product in products) >=1
                              for plant in plants), name="Plant Make")
product_made_by = model.addConstrs((gp.quicksum(plant_make_product[plant, product]
                                         for plant in plants) >=1
                              for product in products), name="Product Made By")

In [24]:
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 144 columns and 72 nonzeros
Model fingerprint: 0x988a6015
Variable types: 108 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 652.0000000
Presolve removed 0 rows and 111 columns
Presolve time: 0.01s
Presolved: 12 rows, 33 columns, 66 nonzeros
Found heuristic solution: objective 403.0000000
Variable types: 0 continuous, 33 integer (33 binary)
Found heuristic solution: objective 347.0000000

Root relaxation: objective 3.140000e+02, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [25]:
model.printAttr('X')


    Variable            X 
-------------------------
Plant Production[Akron,Sedan]            1 
Plant Production[Buffalo,Coupe]            1 
Plant Production[Columbus,SUV]            1 
Plant Production[Detroit,Van]            1 
Plant Production[Evansville,Truck]            1 
Plant Production[Flint,Compact]            1 
